# Project: cleanning - [candy data]
### this notebook is for educational purpose . in this notebook we will do some cleanning

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">wrangling</a></li>
  
</ul>

<a id='intro'></a>
## Introduction

### Dataset Description

> i will go through candy data 2017 , this data has :
- Internal_ID
- Q1-Going Out?: Are you actually going trick or treating yourself? "has values `Yes or No`"
- Q2-Gender: It has four different options, `Femal ,Male ,Other ,I'd rather not say`
- Q3-Age: Numerical field
- Q4-Country: Text Field, but users have written their own version of the names. Example, for America, there are entries such as USA, us, US, America so we should consider that while cleanning
- Q5-State/Province: Text Field, but users have written their own version of the names. Same as the country data.
- Q6-Joy Or Despair: All kinds of chocolate bars are the questions with three distinct options to choose from (Joy, Meh, Despair).
- Q7-Joy Other: Text Field to write items not included above that give you JOY. Lots of missing values.
- Q8-Despair Other: Text Field ti write items not included above that give you DESPAIR. Lots of missing values.
- Q9-Other Comments: Text Field. Lots of missing values.
- Q10-Dress: Binary field. Missing values present
- Unnamed: 113
- Q11-Day: Binary Answer Field. Missing values present


# 1- import packages and load data

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
df = pd.read_excel("candyhierarchy2017.xlsx")

FileNotFoundError: ignored

<a id='wrangling'></a>
## Data Wrangling



### General Properties

# 2- show some general properties for the data

In [ ]:
df.info()
df.shape
df.dtypes
df.columns
df.describe()
df.duplicated().sum()

NameError: ignored

# 3- write summary about what you get form the above functions
> #### tips : this summary can be about : if there is any missing values , if there a wrong data type , if there a wrong range of values , ....

In [ ]:
noticed alot of missing values and the first row consist of all null values that need to be fill, the age column need to be changed to integer , column names arent helpful and need to be changed to easier format to be ease the cleaning process,some columns
doesnt seem to be imporant like unnammed and click coordinates

# cleaning

### 4- drop all the useless columns and  rows with too much NaN ,
> tip : useless columns can be like `Internal ID`and can be also columns with too much NaN
- rename columns for easy access

In [ ]:
df.drop('Unnamed: 113',axis=1,inplace=True)
df.drop('Click Coordinates (x, y)',axis=1,inplace=True)
df.drop([0])

### 5 - clean `going out` column
> tip : you should choose which best value to put in place of these NaNs ( mode, unknown,other ....etc)

In [ ]:
df['Q1: GOING OUT?'].unique()
modee_Q1=df['Q1: GOING OUT?'].mode()
df['Q1: GOING OUT?'].fillna(modee_Q1[0],inplace=True)


### 6 - clean ` gender` column
> tip : you should choose which best value to put in place of these NaNs ( mode, unknown,other ....etc)

In [ ]:
df['Q2: GENDER'].unique()
df['Q2: GENDER'].fillna("I'd rather not say",inplace=True)

### 7 - clean ` age` column
> tip : note the type of age column , and NaN values
 - try to use `to_numeric`
 - tip : you should choose which best value to put in place of these NaNs ( mean, median, ....etc)

In [ ]:
df['Q3: AGE'].unique()
df['Q3: AGE']=pd.to_numeric(df['Q3: AGE'], errors ='coerce')
meann=df['Q3: AGE'].mean() ##will use it to replace strange values with the average age
df['Q3: AGE'].replace(0,meann,inplace=True)
df.loc[ df['Q3: AGE'] > 100 , 'Q3: AGE'] = meann



### 8 - clean ` country` column
> tip : note that country column has country names with different writing ways
  - use fuzzywuzzy to deal with this problem
  - you should choose which best value to put in place of these NaNs ( mode, unknown,other ....etc)

In [ ]:
df['Q4: COUNTRY'].unique()
countries=pd.DataFrame({"country":["USA","United states","America","UK","United kingdom","Canada","UAE","Mexico","France","Finland","Netherlands",
                     "Germany","Costa Rica","Australia","Greece","Ireland","Korea","Japan","South Africa","Iceland","Switzerland","Scotland",
                     "Denmark","Spain","Indonesia","Netherlands","Atlantis","Singapore","Hong Kong","Taiwan"]})
df['Q4: COUNTRY'].fillna("0",inplace=True)
def find_best_match(value, choices):
    if pd.isnull(value):
        return value
    return process.extractOne(str(value), choices)[0]
df['Q4: COUNTRY'] = df['Q4: COUNTRY'].apply(lambda x: find_best_match(x, countries['country']))
df['Q4: COUNTRY'].replace("USA","United states",inplace=True)
df['Q4: COUNTRY'].replace("America","United states",inplace=True)
df['Q4: COUNTRY'].replace("UK","United kingdom",inplace=True)

### 9 - clean ` area` column
> tip : area column has the same problem as country columns , but looking at all it's unique values may give us another way to deal with it rather than try to fix it

In [ ]:
df.rename(columns = {'Q5: STATE, PROVINCE, COUNTY, ETC':'Q5:Area'}, inplace = True)
df["Q5:Area"].unique()
##sice many values doesn't have a meaning and can't be determined ,its would be suffiecient  to only use information from country column and drop this one
df.drop("Q5:Area",axis=1,inplace=True)

### 10 - clean ` Q6` columns
>- tip : Q6 has all kinds of chocolate bars and questions with three distinct options to choose from (Joy, Meh, Despair).
- so all the column that has Q6 is a candy name , so you should go through all these column and you should choose which best value to put in place of these NaNs ( mode, unknown,other ....etc)

In [ ]:
for j in df:
     if j[1]=='6':
        modee=df[j].mode()
        df[j].fillna(modee[0],inplace=True)

### 11 - clean ` dress` column
>tip : you should choose which best value to put in place of these NaNs ( mode, unknown,other ....etc)

In [ ]:
df["Q10: DRESS"].isna().sum()
df["Q10: DRESS"].unique()
modee_dress=df["Q10: DRESS"].mode()
df["Q10: DRESS"].fillna(modee_dress[0],inplace=True)


### 12 - clean ` day` column
>tip : you should choose which best value to put in place of these NaNs ( mode, unknown,other ....etc)

In [ ]:
df['Q11: DAY'].unique()
modee_day=df['Q11: DAY'].mode()
df['Q11: DAY'].fillna(modee_day[0],inplace=True)

### 13 - check
> after finish cleaning you should make sure your data is clean and that :
- there is no wrong data type
- there is no NaN values
- all the column are cleaing and ready for the next step `EDA`

### 14 - write a summary about all the cleaning steps

In [ ]:
#most of the cleaning process depended on the replacing of nan values and it was done either by getting the mode(frequently used value) and using it or looking through unique values of column and using one.
##the age column needed to be changed to another datatype (float) and replacing null values and nonlogical ages with the mean as it is the most conveniant
##the country column was cleaned by filling all null values with any value and creating a list with the true value ,giving a function that check the similarities and replace it
##since some countries its shortcut can also be used , i replaced the shortcut with fullname
##columns that are merged to one question: a loop was used to check the name of each column and perfom the function on it
##some columns was dropped as it had no meaning